In [1]:
def isISBN13(code):
    if not isinstance(code, str):
        return False
    if len(code) != 13:
        return False
    if not code.isdigit():
        return False
    o = sum(int(code[i]) for i in range(0, len(code[:-1]), 2))
    e = sum(int(code[j]) for j in range(1, len(code[:-1]), 2))
    x13 = (10 - (o + 3*e)%10)%10
    
    return x13 == int(code[-1])

def remove_tags(s):
    s = s.strip()
    while s.find('<') >= 0:
        start = s.find('<')
        stop = s.find('>')
        if stop == -1:
            stop - len(s)
        s = s[:start] + s[stop +1:]
    return s.strip()
    
def display_book_info(isbn):
    if not isISBN13(isbn):
        print("Wrong ISBN-13 code")
        return
    
    import urllib.request
    url = 'https://pythia.ugent.be/pythia-share/exercises/isbn9/books.php'
    parameters = '?isbn=' + isbn.strip()
    info = urllib.request.urlopen(url + parameters)

    for line in info:
        line = line.decode('utf-8')
        if line.startswith('<Title>'):
            print('Title: {}'.format(remove_tags(line)))
        if line.startswith('<AuthorsText>'):
            print('Authors: {}'.format(remove_tags(line).rstrip(', ')))
        if line.startswith('<PublisherText'):
            print('Publisher: {}'.format(remove_tags(line)))
    info.close()

In [2]:
def mass_table(file_path):
    amino_acid_masses = {}
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.split()
            amino_acid_masses[parts[0]] = float(parts[1])
    return amino_acid_masses

def protein_mass(sequence, table, peptide=False):
    if peptide:
        return sum([table[prot] for prot in sequence])
    else:
        return sum([table[prot] for prot in sequence]) +  18.01056

In [20]:
def mean(list_scores):
    if all(score is None for score in list_scores):
        return None
    valid_scores = [score for score in list_scores if score is not None]
    return sum(valid_scores) / len(valid_scores)

def database(file_path):
    country_dict = {}
    with open(file_path, 'r') as file:
        for line in file:
            list_country = line.split('\t')
            list_country = [x if x != '' else None for x in list_country]
            country_dict[list_country[1]] = [None if x == None or x == '\n' else int(x) for x in list_country[2:]]
    return country_dict

def oceanHealthIndex(country, data):
    scores = data[country]
    average_5_6 = (scores[5] + scores[6]) / 2 if scores[5] + scores[6] != None else None
    average_8_9 = (scores[8] + scores[9]) / 2 if scores[8] + scores[9] != None else None
    average_11_12 = (scores[11] + scores[12]) / 2 if scores[11] + scores[12] != None else None
    scores = scores[:5] + [mean(scores[5:7])] + scores[7:8] + [average_8_9] + scores[10:11] + [average_11_12]
    return round(mean(scores))

In [ ]:
def atomicmass(file_path):
    mass_dict = {}
    counter = 0
    with open(file_path, 'r') as file:
        for line in file:
            if counter == 0:
                counter +=1
                continue
            elem= line.split('\t')
            mass_dict[elem[1]] = float(elem[4])
    return mass_dict

def molecularmass(formula, index):
    transformed_list = []
    for element in formula.split('-'):
        for i, char in enumerate(element):
            if char.isdigit():
                transformed_list.append(element[:i])
                transformed_list.append(int(element[i:]))
                break
        else:
            transformed_list.append(element)
            transformed_list.append(1)
    
    return sum(j * index[i] for i, j in zip(transformed_list[::2], transformed_list[1::2]))

In [9]:
def reference_spectra(file_path):
    atom_dict = {}
    with open(file_path, 'r') as file:
        for line in file:
            list_atoms = line.split('\t')
            list_atoms = [list_atoms[0]] + list_atoms[1].split(',')
            atom_dict[list_atoms[0]] = tuple(sorted([float(x) for x in list_atoms[1:]]))
    return atom_dict

def reference_lines(m_spectrum, r_spectrum, eps=0.1):
    l_common = []
    for g_m in m_spectrum:
        for g_r in r_spectrum:
            if abs(g_r - g_m) <= eps and g_r not in l_common:
                l_common.append(g_r)
    return len(l_common)

def decomposition(spectrum, reference_dict, eps=0.1, minimum=None):
    decompos = []
    for atom in reference_dict:
        if minimum is None and len(reference_dict[atom]) == reference_lines(spectrum, reference_dict[atom], eps):
            decompos.append(atom)
        elif minimum is not None and reference_lines(spectrum, reference_dict[atom], eps) >= minimum:
            decompos.append(atom)
    return sorted(decompos)

In [ ]:
import math
def euclidean_distance(x,y):
    return math.sqrt((x[0] - y[0])**2 + (x[1] - y[1])**2)
def manhattan_distance(x,y):
    return float(abs(x[0] - y[0]) + abs(x[1] - y[1]))
def chessboard_distance(x,y):
    return float(max(abs(x[0] - y[0]), abs(x[1] - y[1])))

def herd(file_path):
    herd_dict = {}
    with open(file_path, 'r') as file:
        for row, line in enumerate(file):
            for col, char in enumerate(line.strip()):  # Removes '\n'
                if char != '.':
                    herd_dict[(row, col)] = char
    return row + 1, col + 1, herd_dict

def nearest_antelope(pos, herd_dict, distance=euclidean_distance):
    nearest = set()
    curr_dist = float('inf')
    for p, antelope in herd_dict.items():
        dist = distance(pos, p)
        if dist < curr_dist:
            nearest.clear()
            nearest.add(antelope)
            curr_dist = dist
        elif dist == curr_dist:
            nearest.add(antelope)
    return nearest

def regions(file_path, distance=euclidean_distance):
    rows, cols, herd_dict = herd(file_path)
    str_res = [['.'] * cols for _ in range(rows)]
    for (row, col), antelope in herd_dict.items():
        str_res[row][col] = antelope
    for i in range(rows):
        for j in range(cols):
            if str_res[i][j] == '.':
                nearest = nearest_antelope((i, j), herd_dict, distance)
                str_res[i][j] = sorted(nearest)[0].lower()

    return "\n".join("".join(row) for row in str_res)